# Setup

## Verify we're in the Conda environment

In [ ]:
import sys
print(sys.executable)

## Import python packages

In [ ]:
import os
import sys
import json
import openai
from PIL import Image
import base64
import io
from dotenv import load_dotenv
import requests
from openai import OpenAI
import pprint
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

## openAI API key

In [3]:
# Set up your OpenAI API key
# api_key = os.environ.get("OPENAI_API_KEY")

# Load the .env file
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


# Helper functions

## Function to base64 encode an image

In [4]:
def encode_image(image_path):
    """Encode the image to base64 format to send to OpenAI."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

## Function to load existing results from JSON

In [ ]:
def load_existing_results(filename):
    """Load existing data from JSON file if it exists."""
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

## Function to send the request to OpenAI API

In [ ]:
# Function to send the request to OpenAI API
def get_image_description_and_prompts(prompt, base64_image):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        max_tokens=10000
    )

    if (chat_completion.choices[0].finish_reason != "stop"):
        print("Something went wrong during openAI call - finish_reason is not 'stop' ")
        print(chat_completion)
        sys.exit(1)

    content = chat_completion.choices[0].message.content.strip()
    # if content.startswith('```json') and content.endswith('```'):
    #     # Remove the ```json and ``` from the string
    #     cleaned_result = content.strip('```json').strip('```')

    # Convert the content to a dictionary
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        print("Error: The content is not valid JSON.")
        sys.exit()

    pprint.pprint(data)
    return data

## Function to process N images

In [ ]:
# Function to process N images
def process_images(prompt, output_file):
    results = []

    # If output file exists, load existing data
    if os.path.exists(output_file):
        with open(output_file, 'r') as f:
            try:
                results = json.load(f)
            except json.JSONDecodeError:
                print(f"Warning: Could not parse existing data in {output_file}. Starting fresh.")
    
    # Create a set of image paths already processed
    existing_image_paths = {entry["image"] for entry in results}

    # Base directory containing all the folders
    base_dir = Path("/Scandisk/onicai/charles/images")
    image_paths = [path for path in base_dir.glob("**/*.png") if not path.name.startswith(".")]
    thumbnail_size = (100, 100)

    icount = 0
    for image_path in image_paths:
        icount += 1
        print(f"------------------\n Processing image {icount}: {image_path}")
        # Open and display the image
        image = Image.open(image_path)
        image.thumbnail(thumbnail_size)  # Resize the image to a thumbnail
        plt.figure(figsize=(2, 2))  # Adjust figure size
        plt.imshow(image)
        plt.axis('off')  # Hide axes for better view
        plt.show()

        # Check if this image has already been processed
        if str(image_path) in existing_image_paths:
            print("This image was already processed, with these results:")
            existing_entry = next(entry for entry in results if entry["image"] == str(image_path))
            print(json.dumps(existing_entry["response"], indent=4))
            continue
        
        # Not yet processed, go ahead and call openAI
        print("This image was not yet processed. Calling openAI...")
        if os.path.exists(image_path):
            base64_image = encode_image(image_path)
            response = get_image_description_and_prompts(prompt, base64_image)
            results.append({
                "image": str(image_path),
                "response": response
            })
            print(json.dumps(response, indent=4))
        else:
            print(f"Image {image_path} does not exist.")
            sys.exit()
    
        # Save the updated results back to the output file
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=4)

# Run it

In [ ]:
output_file = './1-create-prompts-with-openai.json'
num_stories = 20

prompt = """
First, describe this image of a Teddy Bear named Charles.

Then, create 20 opening sentences for a short children's story that will capture the description of this image.

Make sure to include the specific props and items shown in the image in the opening sentences.

The prompts must be shorter than 15 words.

Return the result in JSON with this schema:
{
        "image": "...png",
        "response": {
            "description": "...",
            "opening_sentences": [
                "...",
                "...",
                ...
            ]
        }
    }
"""

process_images(prompt, output_file)